In [1]:
import os
import traceback
import sys


In [2]:
from dotenv import load_dotenv

In [3]:
from langchain_ollama import ChatOllama
from langchain.prompts import ChatPromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain.chains import RetrievalQA



e:\ChatwithFile\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from pinecone import Pinecone

In [5]:
from PyPDF2 import PdfReader

In [6]:
file_path=r"E:\ChatwithFile\Einstein_Relativity.pdf"

In [7]:
reader= PdfReader(file_path)
text = ""
for page in reader.pages:
    text += page.extract_text() + "\n"
    

In [8]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_text(text)

In [9]:
len(texts)

265

In [10]:
load_dotenv()

True

In [1]:
PINECONE_API_KEY =""
PINECONE_INDEX =""

In [12]:
pc = Pinecone(api_key=PINECONE_API_KEY)

In [13]:
index=pc.Index(PINECONE_INDEX)

In [14]:
embedding= OllamaEmbeddings(model="llama3.1:8b")

In [24]:
vector_store = PineconeVectorStore.from_texts(texts=texts,embedding=embedding, index_name="chatwithfile")

In [25]:
llm= ChatOllama(model="llama3.1:8b")

In [26]:
prompt = ChatPromptTemplate.from_template(
    """Answer the question based only on the following context: {context}

    Question: {question}
    Answer:"""
)

In [27]:
retriever = vector_store.as_retriever()

In [28]:
from langchain_core.runnables import RunnableMap, RunnablePassthrough

In [ ]:
rag_chain = (
    RunnableMap({
        "context": retriever,
        "question": RunnablePassthrough()
    })
    | prompt
    | llm
)

In [32]:
result = rag_chain.invoke("What is the special theory of relativity?")

In [33]:
print(result.content)

The special theory of relativity, according to Document(id='a8908772-59f7-427d-89eb-65cc87ba1fa5', has reference to Galileian domains, i.e. to those in which no gravitational field exists. In this connection a Galileian reference-body serves as body of reference, i.e. a rigid body the state of motion of which is so chosen that the Galileian law of the uniform rectilinear motion of “isolated” material points holds relatively to it.
